In [1]:
import cv2
from deepface import DeepFace
import numpy as np
from pytube import YouTube
import os
import tempfile
import json

class FaceAnalyzer:
    def __init__(self, detector_backend='retinaface'):
        self.valid_backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe', 'yolov8', 'yunet']
        if detector_backend not in self.valid_backends:
            raise ValueError(f"Invalid detector backend. Choose from {self.valid_backends}")
        
        self.detector_backend = detector_backend
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    def deepface_analysis(self, frame):
        return DeepFace.analyze(frame, actions=['age', 'gender', 'race', 'emotion'], enforce_detection=False, detector_backend=self.detector_backend)

    def detect_faces(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        return self.face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)



class RealTimeAnalyzer:
    def __init__(self, video_source=0, frame_skip=5):
        self.cap = cv2.VideoCapture(video_source)
        self.face_analyzer = FaceAnalyzer(detector_backend='yunet')
        self.frame_skip = frame_skip
        self.sidebar_frame = None  # New attribute to store the last sidebar frame

    def process_frames(self):
        
        faces_dir = "detected_faces"
        if not os.path.exists(faces_dir):
            os.makedirs(faces_dir)
            
        face_info_dict = {}
        face_id = 0
        
        frame_count = 0
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            faces = self.face_analyzer.detect_faces(frame)
            
            num_faces = len(faces)
            cv2.putText(frame, f"Faces Detected: {num_faces}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            frame_skip = 5  # Skip every 5 frames
            if frame_count % frame_skip == 0:
                sidebar_info = []

                for idx, (x, y, w, h) in enumerate(faces):
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # blue color for the rectangle
                    cropped_face = frame[y:y+h, x:x+w]
                     # Save face image
                    face_path = os.path.join(faces_dir, f"face_{face_id}.jpg")
                    cv2.imwrite(face_path, cropped_face)
                    
                    
                    try:
                        results = self.face_analyzer.deepface_analysis(cropped_face)
                        analysis = results[0] if isinstance(results, list) and len(results) > 0 and isinstance(results[0], dict) else {}
                        # Store analysis info with face_id as key
                        if analysis:
                            face_info_dict[face_id] = analysis

                        face_id += 1
                        dominant_emotion = analysis.get('dominant_emotion', 'N/A')
                        age = analysis.get('age', 'N/A')
                        gender = analysis.get('gender', 'N/A')
                        race = analysis.get('dominant_race', 'N/A')
                        sidebar_info.append(f"ID: {idx+1} | Emotion: {dominant_emotion} | Age: {age} | Gender: {gender} | Race: {race}")

                    except Exception as e:
                        print('Error:', e)

                # Update the sidebar frame if there's new data
                if sidebar_info:
                    self.sidebar_frame = self._create_sidebar(sidebar_info)

            if self.sidebar_frame is not None:
                cv2.imshow('Face Analysis Info', self.sidebar_frame)

            frame_count += 1
            cv2.imshow('Camera Feed', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()

    def _create_sidebar(self, sidebar_info):
        width, height = 1000, 200  # Adjust dimensions as needed
        sidebar_frame = 255 * np.ones((height, width, 3), dtype=np.uint8)  # white background

        y0, dy = 25, 25
        for idx, info in enumerate(sidebar_info):
            y = y0 + idx * dy
            cv2.putText(sidebar_frame, info, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)  # Black text

        return sidebar_frame


def get_youtube_video(youtube_link):
    yt = YouTube(youtube_link)
    ys = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

    # Save the video with its title in the current working directory
    output_path = os.path.join(os.getcwd(), ys.default_filename)
    if not os.path.exists(output_path):  # Only download if it doesn't exist
        ys.download(output_path=output_path)
    return output_path


if __name__ == "__main__":
    print("Choose an option:")
    print("1. Use Camera")
    print("2. Provide Local Path or YouTube Link for Video (mp4 only)")
    choice = input("Enter your choice (1/2): ")

    video_source = 0 # default to camera

    if choice == "2":
        video_path_or_link = input("Please provide the video link (YouTube) or local path (mp4 only): ")
        
        if "youtube.com" in video_path_or_link or "youtu.be" in video_path_or_link:
            print("Downloading YouTube video, please wait...")
            video_source = get_youtube_video(video_path_or_link)
            print(f"Download finished. Saved at {video_source}")
        elif not video_path_or_link.lower().endswith('.mp4'):
            print("Invalid format. Only .mp4 files are accepted.")
            exit()
        else:
            video_source = video_path_or_link

    analyzer = RealTimeAnalyzer(video_source)
    analyzer.process_frames()

2023-12-07 13:43:57.912021: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Choose an option:
1. Use Camera
2. Provide Local Path or YouTube Link for Video (mp4 only)


QObject::moveToThread: Current thread (0x67ef6f0) is not the object's thread (0x5d36090).
Cannot move to target thread (0x67ef6f0)

QObject::moveToThread: Current thread (0x67ef6f0) is not the object's thread (0x5d36090).
Cannot move to target thread (0x67ef6f0)

QObject::moveToThread: Current thread (0x67ef6f0) is not the object's thread (0x5d36090).
Cannot move to target thread (0x67ef6f0)

QObject::moveToThread: Current thread (0x67ef6f0) is not the object's thread (0x5d36090).
Cannot move to target thread (0x67ef6f0)

QObject::moveToThread: Current thread (0x67ef6f0) is not the object's thread (0x5d36090).
Cannot move to target thread (0x67ef6f0)

QObject::moveToThread: Current thread (0x67ef6f0) is not the object's thread (0x5d36090).
Cannot move to target thread (0x67ef6f0)

QObject::moveToThread: Current thread (0x67ef6f0) is not the object's thread (0x5d36090).
Cannot move to target thread (0x67ef6f0)

QObject::moveToThread: Current thread (0x67ef6f0) is not the object's thread

Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invalid detector_backend passed - yunet
Error: invali

In [1]:
import cv2
from mtcnn import MTCNN

# Initialize the MTCNN face detector
detector = MTCNN()

# Open the default camera
cap = cv2.VideoCapture(1)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # If frame is read successfully, proceed
    if ret:
        # Detect faces in the frame
        faces = detector.detect_faces(frame)
        
        # Draw bounding boxes around detected faces
        for face in faces:
            x, y, w, h = face['box']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Display the frame with detected faces
        cv2.imshow("Face Detection", frame)

    # If 'q' is pressed, exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()


2023-12-07 11:38:38.994086: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 11:38:40.386118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 11:38:40.402722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#

1/1 [==============================] - 0s 123ms/step


2023-12-07 11:38:42.031484: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 13ms/step


QObject::moveToThread: Current thread (0x831ee80) is not the object's thread (0x8566db0).
Cannot move to target thread (0x831ee80)

QObject::moveToThread: Current thread (0x831ee80) is not the object's thread (0x8566db0).
Cannot move to target thread (0x831ee80)

QObject::moveToThread: Current thread (0x831ee80) is not the object's thread (0x8566db0).
Cannot move to target thread (0x831ee80)

QObject::moveToThread: Current thread (0x831ee80) is not the object's thread (0x8566db0).
Cannot move to target thread (0x831ee80)

QObject::moveToThread: Current thread (0x831ee80) is not the object's thread (0x8566db0).
Cannot move to target thread (0x831ee80)

QObject::moveToThread: Current thread (0x831ee80) is not the object's thread (0x8566db0).
Cannot move to target thread (0x831ee80)

QObject::moveToThread: Current thread (0x831ee80) is not the object's thread (0x8566db0).
Cannot move to target thread (0x831ee80)

QObject::moveToThread: Current thread (0x831ee80) is not the object's thread

1/1 [==============================] - 0s 12ms/step
